In [2]:
import pandas as pd
import numpy as np
import nltk
import re
from datetime import date, datetime
import time
import matplotlib.pyplot as plt
import urllib.request
from tqdm import tqdm
import pickle
import spacy
import random
import ast
import seaborn as sns
import networkx as nx

In [3]:
tqdm.pandas()

In [4]:
pd.set_option('display.max_colwidth', -1)

In [5]:
# getting the burst timeline..
df_tweets = pd.read_csv('/home/auppal8/HexagonData_new.csv',lineterminator='\n', index_col=None)

In [6]:
df_timeline = pd.read_csv('/home/auppal8/userTimelineData.csv',lineterminator='\n',)

In [7]:
df_following = pd.read_excel('/home/auppal8/updatedfollowingData1117.xlsx', index_col=0)

In [8]:
# following data for user...
df_300 = pd.read_csv("~/Following300_1220.csv",index_col=0)

In [9]:
#no of hashtags per tweet
# url and then no of url per tweet
# no of @user-mention per tweet
# percentage of retweet per user (user retweets / total no of tweets)

In [10]:
def hashtag_count(df):
    if df is not np.nan:
        hashtags = ast.literal_eval(df)
        if (hashtags is not None):
            return (len(hashtags))
        else:
            return(0)
    else:
        return(0)

In [11]:
# count no of hashtags
df_timeline['hashtags'] = df_timeline['hashtags'].progress_apply(hashtag_count)

100%|██████████| 1344182/1344182 [00:10<00:00, 128569.39it/s]


In [208]:
# find the occurence of urls
df_timeline['urls_id'] = df_timeline['tweetText'].str.findall(r'(https?://\S+)')

In [12]:
# count the no of urls per tweet
df_timeline['urls'] = df_timeline['tweetText'].str.count(r'(https?://\S+)')

In [13]:
# count no of user mentiosn per tweet
df_timeline['author_mentions'] = df_timeline['tweetText'].str.count(r'(\@\w+)')

In [26]:
# just to get avergae user mentions per user
df_user = df_timeline.groupby(df_timeline['userID']).agg({'author_mentions': 'median',
                                                          'urls' : 'median',
                                                          'retweetCount' : 'sum'})

In [33]:
total = df_user['retweetCount'].sum()
df_user['retweetCount'] = df_user['retweetCount'].apply(lambda x: (x/total) * 100)

In [41]:
df_timeline.columns

Index(['favourites_count', 'friendList', 'friendsCount', 'hashtags',
       'imageurl', 'lang', 'retweetCount', 'retweeted', 'tweetCreatedAt',
       'tweetId', 'tweetText', 'userCreatedAt', 'userDescription',
       'userFollowersCount', 'userID', 'userLocation', 'userName', 'urls',
       'author_mentions'],
      dtype='object')

In [40]:
# start with behavious characteristics
# max, min and average no of tweets per day  --done
# max, min time , average time interval between consecutive tweets --done
# total no of tweets
# no of unique urls

In [48]:
df_timeline['tweetCreatedAt'] = pd.to_datetime(df_timeline.tweetCreatedAt)

In [89]:
df_tweets = df_timeline.groupby([df_timeline['userID'],df_timeline['tweetCreatedAt'].dt.date])['tweetId'].agg('count')


In [92]:
df_tweets = pd.DataFrame(df_tweets)

In [133]:
# avg, min and max tweets
df_user['min_tweets'] = df_tweets.reset_index(level=0).groupby(['userID']).min()
df_user['max_tweets'] = df_tweets.reset_index(level=0).groupby(['userID']).max()
df_user['avg_tweets'] = df_tweets.reset_index(level=0).groupby(['userID']).mean()

In [155]:
df_tweets['diff'] = df_tweets.reset_index(level=0).index

In [159]:
df_tweets['diff'] = abs(df_tweets['diff'] - df_tweets['diff'].shift())

In [191]:
df_tweets['diff'] = df_tweets['diff'].apply(lambda x: x.days)

In [228]:
# min , max , avg no of days in consecutive intervals..and total no of tweets
df_user['min_interval'] = df_tweets.reset_index(level=0).groupby(['userID'])['diff'].min()
df_user['max_interval'] = df_tweets.reset_index(level=0).groupby(['userID'])['diff'].max()
df_user['avg_interval'] = df_tweets.reset_index(level=0).groupby(['userID'])['diff'].mean()
df_user['total_tweets'] = df_tweets.groupby(['userID'])['tweetId'].sum()

In [229]:
df_user['total_urls'] = df_timeline.groupby(['userID']).urls_id.sum().apply(lambda x: len(set(x)))

In [282]:
df_userNames = (df_timeline[['userID','userName','imageurl']].drop_duplicates()).reset_index(drop=True)

In [285]:
df_user =  df_userNames.join(df_user, on='userID')

In [286]:
df_user.to_csv('user_characteristics.csv')

In [287]:
df_user

,userID,userName,imageurl,author_mentions,urls,retweetCount,min_tweets,max_tweets,avg_tweets,min_interval,max_interval,avg_interval,total_urls,total_tweets
0,736972871671644160,Mood,http://pbs.twimg.com/profile_images/741806698260779008/Gpt4pGvP_normal.jpg,0.0,0.0,1.057714e-05,1,15,7.166667,1.0,869.0,147.166667,4,43
1,716196216023941121,♡_.princesseuuh_.♡,http://pbs.twimg.com/profile_images/837052146138902535/uPq1L4Ti_normal.jpg,0.0,1.0,6.942869e-05,1,18,2.197368,1.0,926.0,18.328947,95,167
2,716040514097643522,LoadUp Vape Supply,http://pbs.twimg.com/profile_images/900819405700907008/vUwF7uVE_normal.jpg,1.0,1.0,1.583584e-06,1,9,2.564103,1.0,99.0,2.551282,216,200
3,776658764342493185,Connor Simons🥀,http://pbs.twimg.com/profile_images/966812335850467329/MGuGXCr3_normal.jpg,0.0,1.0,6.671407e-02,1,8,2.044944,1.0,219.0,4.910112,107,182
4,854000327200309252,GN CoalitionforYouth,http://pbs.twimg.com/profile_images/854374689589780481/BnEyo70h_normal.jpg,0.0,1.0,8.286425e-06,1,13,2.941176,1.0,23.0,2.985294,218,200
5,909804429133545477,KLOWDZ Vapor & Smoke,http://pbs.twimg.com/profile_images/912765220568920074/TBWuH-hb_normal.jpg,0.0,1.0,7.170945e-07,1,12,1.801802,1.0,149.0,2.675676,201,200
6,932089278389157893,Everything4LessStore,http://pbs.twimg.com/profile_images/932090112464134145/07MXvsHX_normal.jpg,1.0,1.0,0.000000e+00,1,47,9.523810,1.0,317.0,19.857143,200,200
7,907292356952686592,Eliza,http://pbs.twimg.com/profile_images/965967582414168067/F2HgPDJA_normal.jpg,0.0,0.0,4.726748e-04,1,16,7.142857,1.0,191.0,8.178571,60,200
8,973253202308038664,Dr. Mark Rubinstein,http://pbs.twimg.com/profile_images/973255200025620480/Yua9JYTp_normal.jpg,0.0,1.0,1.152331e-05,1,6,1.714286,1.0,217.0,11.250000,61,48
9,859245303483858953,Maya Burns,http://pbs.twimg.com/profile_images/1048446111206137856/_6N6xU5I_normal.jpg,0.0,1.0,5.660200e-02,5,37,15.307692,1.0,74.0,6.615385,150,199
